In [48]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from sklearn.preprocessing import StandardScaler #divisão de dados e pré-processamento
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import pysr
pysr.silence_julia_warning()
import json
import os
import dill as pickle
import time



In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [50]:
class MyDataset():
 
  def __init__(self, data):

    data = data.values
 
    x = data[:, 0:-1]
    sc = StandardScaler() #dimensionamos nosso conjunto de dados #mesma ordem ñ precisa
    x_sc = sc.fit_transform(x)
    
    y = data[:, -1]
 
    self.x_train=torch.tensor(x_sc, dtype=torch.float32) #convertendo nosso conjunto de dados em tensores de tocha.
    #self.x_train=torch.tensor(x, dtype=torch.float32)
    self.x_train = self.x_train.to(device)
    
    self.y_train=torch.tensor(y, dtype=torch.float32)
    self.y_train = self.y_train.to(device)
    
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],  torch.unsqueeze(self.y_train[idx], dim=0) #Retorna um novo tensor com uma dimensão de tamanho um inserido na posição especificada.

    #funcionalidade literal de obter um único ponto de dados por índice e retornar o comprimento do conjunto de dados.



In [58]:
def read_data(data_path, sep=','):
    print("Loading the data ...")
    file_extension = pathlib.Path(data_path).suffix

    if file_extension == '.txt':
        data = np.loadtxt(data_path)
        assert isinstance(data, np.ndarray), f"The input file must be a numpy array but is {type(data)}"
        col_names = ['X'+str(i+1) for i in range(data.shape[1])]
        data = pd.DataFrame(data, columns=col_names, dtype=np.float32)
    elif file_extension == '.csv':
        data = pd.read_csv(data_path, sep=sep, dtype=np.float32)
    else:
        print(f"the type of the data is {type(file_extension)} but a csv or saved numpy array is expected")
        raise TypeError()
    print("Reading the data is done!")
    return data

In [59]:
data = read_data('example2.txt')

Loading the data ...
Reading the data is done!


In [60]:
myDs=MyDataset(data)
train_loader=DataLoader(myDs, batch_size=2048, shuffle=True)

In [61]:
def network(input_dimension,hidden_dimension, output_dimension):

  print("Constructing the neural network...")

  modules=[]
  modules.append(torch.nn.Linear(input_dimension, hidden_dimension[0]))
  modules.append(torch.nn.ReLU()) #função de ativação
  for i in range(len(hidden_dimension)-1):
    modules.append(torch.nn.Linear(hidden_dimension[i], hidden_dimension[i+1]))
    modules.append(torch.nn.ReLU())

  modules.append(torch.nn.Linear(hidden_dimension[-1], output_dimension))

  net = torch.nn.Sequential(*modules) #incializar módulo interno

  if torch.cuda.is_available():
    net = net.to(device)
  
  print("Construction is done!")
  
  return net


In [68]:
dim_features = data.shape[1] - 1
net= network(dim_features, [200,200,200, 100, 100], 1)

Constructing the neural network...
Construction is done!


In [67]:
def train(net, train_loader, epochs=100, verbose=1):

  print("Training the neural network with the data...")
  n_show_epoch = int(epochs/10)

  loss_func = nn.MSELoss()


  optimizer = torch.optim.Adam(net.parameters()) #Gradient Descent
  scheduler = StepLR(optimizer, step_size=int(epochs/10), gamma=0.1, verbose=False)


  for e in range(epochs):
  
    running_loss= 0

    for features,labels in train_loader:
        
        output= net(features) 

        loss= loss_func(output, labels)
        optimizer.zero_grad() #zerar gradientens - não se quer acumulação
        loss.backward() #cálculo do gradiente de perda
        
        optimizer.step() #atualiza os parâmentros
        scheduler.step()
        

        running_loss += loss.item ()
    if verbose and e % n_show_epoch==0:
      print(f"epoch: {e}/{epochs}, loss: {loss.item()}")

  print(f"Training the neural network is done! The final loss is {loss.item()}")  
  
  return net


In [69]:
model = train(net, train_loader, epochs=2000, verbose=1)

Training the neural network with the data...
epoch: 0/2000, loss: 39.24441146850586
epoch: 200/2000, loss: 10.462998390197754
epoch: 400/2000, loss: 4.621273517608643
epoch: 600/2000, loss: 5.732011318206787
epoch: 800/2000, loss: 9.236776351928711
epoch: 1000/2000, loss: 6.360742092132568
epoch: 1200/2000, loss: 7.206399440765381
epoch: 1400/2000, loss: 7.268052101135254
epoch: 1600/2000, loss: 5.322818279266357
epoch: 1800/2000, loss: 5.892086505889893
Training the neural network is done! The final loss is 3.627647876739502


In [60]:
def check_translational_symmetry_plus(model, data, min_error=0.05, shift=None, verbose=1):

    print("Checking the translational symmetry by plus...")
    data_translated = torch.from_numpy(data.values).to(device)
    #print(data_translated[:5, :])
    num_variables = data_translated.size()[1] - 1
    y = torch.unsqueeze(data_translated[:, -1], dim=1)

    if os.path.exists('columns.pkl'):
        columns = pickle.load("columns.pkl")
    else:
        columns = []
        


    with torch.no_grad():            
        
        for i in range(num_variables):
            for j in range(i+1, num_variables):
                #if i<j:
                x = data_translated[:, 0:-1].clone()
                #print(data_translated[:5, :])


                if shift is None:
                    shift = 0.5 * min(torch.std(x[:,i]),torch.std(x[:,j]))

                x[:,i] += shift #(x[:,i] + shift).clone()
                x[:,j] += shift #(x[:,j] + shift).clone()
                errors = abs(y - model(x))
                error = torch.median(errors)
                if verbose:
                    print(f"Columns {(i+1,j+1)} -> error {error}")
                if error < min_error:
                    #columns[(i+1,j+1)] = float(error)
                    columns.append([i+1, j+1])
    with open('columns.pkl', 'wb') as f:
        pickle.dump(columns, 'columns.pkl')
        
    print("Checking the translational symmetry is done!")
    
    #return columns

In [61]:
columns = check_translational_symmetry_plus(model, data, min_error=0.05)

Checking the translational symmetry by plus...
Columns (1, 2) -> error 0.007878899574279785
Columns (1, 3) -> error 0.5623359680175781
Columns (1, 4) -> error 0.5594482421875
Columns (2, 3) -> error 0.562767505645752
Columns (2, 4) -> error 0.5608224868774414
Columns (3, 4) -> error 0.00790262222290039
Checking the translational symmetry is done!


In [ ]:
def check_translational_symmetry_didive(model, data, min_error=0.05, shift=None, verbose=1):

    print("Checking the translational symmetry by divide...")
    data_translated = torch.from_numpy(data.values).to(device)
    #print(data_translated[:5, :])
    num_variables = data_translated.size()[1] - 1
    y = torch.unsqueeze(data_translated[:, -1], dim=1)

    if os.path.exists('columns.pkl'):
        columns = pickle.load("columns.pkl")
    else:
        columns = []
        


    with torch.no_grad():            
        
        for i in range(num_variables):
            for j in range(i+1, num_variables):
                #if i<j:
                x = data_translated[:, 0:-1].clone()
                #print(data_translated[:5, :])


                if shift is None:
                    shift = 0.5 * min(torch.std(x[:,i]),torch.std(x[:,j]))

                x[:,i] /= shift #(x[:,i] + shift).clone()
                x[:,j] /= shift #(x[:,j] + shift).clone()
                errors = abs(y - model(x))
                error = torch.median(errors)
                if verbose:
                    print(f"Columns {(i+1,j+1)} -> error {error}")
                if error < min_error:
                    #columns[(i+1,j+1)] = float(error)
                    columns.append([i+1, j+1])
    with open('columns.pkl', 'wb') as f:
        pickle.dump(columns, 'columns.pkl')
        
    print("Checking the translational symmetry is done!")
    
    #return columns

In [62]:
def save_dict(dict_path, obj):
    with open(dict_path, 'w') as convert_file:
        convert_file.write(json.dumps(obj))

def read_dict(dict_path):
    with open(dict_path) as json_file:
        d = json.load(json_file)
    return d

In [72]:
data_new = data.iloc[:, 0:-1].copy()
data_new.columns[-1]

'X4'

In [67]:
def apply_translational_symmetry(data, columns):
    print(f"Reducing the number of indepent variables by {len(columns)}")


    if os.path.exists('variable_change.json'):
        variable_change = read_dict('variable_change.json')
    else:
        variable_change = dict()

    data_new = data.iloc[:, 0:-1].copy()
    for i, j in columns:
        
        if data_new.columns[-1][0] == 'X':
            col_num = 1
        else:
            col_num = int(data_new.columns[-1][1:]) + 1
        data_new['u'+str(col_num)] = data_new.iloc[:, i-1] - data_new.iloc[:, j-1]
        variable_change['u'+str(col_num)] = data_new.columns[i-1] + '-' + data_new.columns[j-1]



    #drop_cols = list(set([i for l in columns.keys() for i in l])) # the list of the columns to be deleted
    #drop_cols = [i-1 for i in drop_cols] #correcting the index of the columns to be deleted
    drop_cols = [i-1 for sublist in columns for i in sublist]

    data_new = data_new.drop(labels=data.columns[drop_cols], axis=1)
    data_new = pd.concat([data_new, data.iloc[:, -1]], axis=1)
    
    data_new.to_csv('data_new.csv', index=False)
    save_dict('variable_change.json', variable_change)

    return data_new, variable_change
  

In [68]:
data_new, variable_change = apply_translational_symmetry(data, columns)

Reducing the number of indepent variables by 2


In [70]:
def SR_GA(path_data, number_of_samples=500, iterations=10):
    data = pd.read_csv(path_data)
    indices = np.random.choice(data.shape[0], number_of_samples, replace=False)
    X = data.iloc[indices, 0:-1]  #separando variáveis de entrada
    y = data.iloc[indices, -1].values

    sr_model = pysr.PySRRegressor(
    niterations=iterations,
    #populations=10,
    binary_operators=["+", "*", "-", "/", "pow"],
    #batching=True,
    #batchSize = 128,
    #procs = 20,
    #multithreading = True,
    unary_operators=[
        "cos",
        "exp",
        "sin",  # Pre-defined library of operators (see docs)
        "inv(x) = 1/x",  # Define your own operator! (Julia syntax)
                    ],
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
    )

    sr_model.fit(X, y)

 
    sr_model.equations.to_csv('results.csv', index=False)
    #sr_model.raw_julia_state = None

    #with open('sr_model.pkl', 'wb') as sr_model_file:
    #    pickle.dump(sr_model, sr_model_file)



    return sr_model



In [71]:
sr_model = SR_GA('data_new.csv', iterations=10)

/home/vahid/miniconda3/lib/python3.9/site-packages/pysr/sr.py:942: UserWarning: Resetting variable_names from X.columns
  warnings.warn("Resetting variable_names from X.columns")
  Activating project at `~/.julia/environments/pysr-0.7.9`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.7.9/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Manifest.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Manifest.toml`


In [ ]:
def show_results(sr_model=None, all_results=False):
    """shows the result of symbolic regression using the PySR

    Args:
        sr_model (PySR model, optional): a fitted PySR model on data. Defaults to None.
        best (bool, optional): True means printing the best model. False means printing all the models. Defaults to True.
    """

    time.sleep(2)
    
    variable_change = read_dict('variable_change.json')
    # the below part for loading isn't working right now
    if sr_model is None:
        with open('sr_model.pk', 'rb') as sr_model_file:
            sr_model = pickle.load(sr_model_file)

    if all_results is False:
        eq = sr_model.sympy()
        eq = eq.subs(variable_change).simplify()
        print("--------------------------\n")
        print("The Final Result is:\n")
        print(eq)
        #return eq
    else:
        print("--------------------------\n")
        print("The Final Result is:\n")
        print(sr_model.equations.replace(variable_change, regex=True).loc[:, ['loss', 'equation']])
        #return sr_model.equations.replace(variable_change, regex=True).loc[:, ['loss', 'equation']]

In [ ]:
show_results(sr_model, best=True)

(Abs(x1 - x2)**1.9997796 + Abs(x3 - x4)**1.9997953)**0.5000427


Essa função abaixo junta todas as funções e executa todas para chegar na resposta final

In [ ]:
def run_regression(data_path, 
                    epochs=100,
                    iterations=10,
                    hidden_layers = [200, 200, 100],
                    all_results=False,
                    min_error=0.01,
                    number_of_samples=500,
                    batch_size = 1028,
                    verbose=1 ):
    """run a neural_genetic symbolic regression on data

    Args:
        data_path (_type_): _description_
        epochs (int, optional): _description_. Defaults to 100.
        iterations (int, optional): _description_. Defaults to 10.
        hidden_layers (list, optional): _description_. Defaults to [200, 200, 100].
        best_result (bool, optional): _description_. Defaults to True.
        min_error (float, optional): _description_. Defaults to 0.05.
        number_of_samples (int, optional): _description_. Defaults to 500.
        batch_size (int, optional): _description_. Defaults to 1028.

    Returns:
        _type_: A fitted model of PySR type
    """
                    
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    data = read_data(data_path)
    myDs=MyDataset(data)
    train_loader=DataLoader(myDs, batch_size=batch_size, shuffle=True)
    dim_features = data.shape[1] - 1
    net= network(dim_features, hidden_layers, 1)
    model = train(net, train_loader, epochs=epochs, verbose=verbose)
    columns = check_translational_symmetry_minus(model, data, min_error=0.05, verbose=verbose)
    data_new, variable_change = apply_translational_symmetry(data, columns)    
    sr_model = SR_GA('data_new.csv', iterations=iterations, number_of_samples=number_of_samples)
    show_results(sr_model)
    
    return sr_model

In [ ]:
run_regression('example1.txt')

# Testar o module `sr_utils`

In [1]:
import sr_utils

In [2]:
model = sr_utils.run_regression('example1.txt')

Loading the data ...
Reading the data is done!
Constructing the neural network...
Construction is done!
Training the neural network with the data...
epoch: 0/100, loss: 0.9433866143226624
epoch: 10/100, loss: 0.0052633136510849
epoch: 20/100, loss: 0.0009353350615128875
epoch: 30/100, loss: 0.0005453507765196264
epoch: 40/100, loss: 0.0005782583029940724
epoch: 50/100, loss: 0.00034297117963433266
epoch: 60/100, loss: 0.00026737406733445823
epoch: 70/100, loss: 0.0003475161793176085
epoch: 80/100, loss: 0.00027960652369074523
epoch: 90/100, loss: 0.0005731609999202192
Training the neural network is done! The final loss is 0.00021734654728788882
Checking the translational symmetry by plus...
Columns (1, 2) -> error 0.006580829620361328
Columns (1, 3) -> error 0.5619543194770813
Columns (1, 4) -> error 0.5637662410736084
Columns (2, 3) -> error 0.5615277290344238
Columns (2, 4) -> error 0.5633257627487183
Columns (3, 4) -> error 0.006571054458618164
Checking the translational symmetry is

/home/vahid/miniconda3/lib/python3.9/site-packages/pysr/sr.py:942: UserWarning: Resetting variable_names from X.columns
  warnings.warn("Resetting variable_names from X.columns")
  Activating project at `~/.julia/environments/pysr-0.7.9`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.7.9/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Manifest.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.7.9/Manifest.toml`


In [73]:
ll = [(1,2), (3,4)]

In [74]:
[i for item in ll for i in item]

[1, 2, 3, 4]

In [4]:
with open("columns.pkl", 'wb') as f:
    pickle.dump(ll, f)

In [5]:
with open('columns.pkl', 'rb') as f:
    lll = pickle.load(f)

In [6]:
lll

[(1, 2), (3, 4)]